In [41]:
import gzip
all_lines = []
#with gzip.open("superfam_all_bacteria_and_archaea_sequences_and_embeddings_test.gz", 'rt') as zip_file:
#with gzip.open("superfam_all_bacteria_and_archaea_sequences_and_embeddings_test.gz", 'rt') as zip_file:
with gzip.open("superfam_16S_bacteria_and_archaea_sequences_and_embeddings_train.gz", 'rt') as zip_file:
    all_lines = zip_file.readlines()
print(len(all_lines))

293700


In [42]:
from random import shuffle
length_index = int(len(all_lines)/3)
all_indexes = list(range(0,length_index))
shuffle(all_indexes)
print(all_indexes[:10])

[32223, 53631, 65059, 20337, 77798, 81366, 53662, 89577, 6159, 94468]


In [43]:
#file_write = gzip.open("randomized_superfam_all_bacteria_and_archaea_sequences_and_embeddings.gz", "wt")
#file_write = gzip.open("randomized_superfam_all_bacteria_and_archaea_sequences_and_embeddings_test.gz", "wt")
file_write = gzip.open("randomized_superfam_16S_bacteria_and_archaea_sequences_and_embeddings_train.gz", "wt")
count = 0
for i in all_indexes:
    idx = i*3
    file_write.write(all_lines[idx])
    file_write.write(all_lines[idx+1])
    file_write.write(all_lines[idx+2])
    count += 1
    #if (count%100 == 0):
     #   break
file_write.close()

In [44]:
# read in the randomized input file and create input for network
# really short for now
import numpy as np
import h5py
alphabet = "NACGT"
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
time_steps = 100
batch_size = 1
category = 10
count = 0
X = []
Y = []
#with gzip.open("randomized_superfam_all_bacteria_and_archaea_sequences_and_embeddings_train.gz", 'rt') as zip_file:
#with gzip.open("randomized_superfam_all_bacteria_and_archaea_sequences_and_embeddings_test.gz", 'rt') as zip_file:
with gzip.open("randomized_superfam_16S_bacteria_and_archaea_sequences_and_embeddings_train.gz", 'rt') as zip_file:
    for header in zip_file:
        #if (count >= 100000):
        #if (count >= 1000):    
         #   break
        #print(line)
        x_data = zip_file.readline().strip()
        y_data = zip_file.readline().strip()
        integer_encoded = [char_to_int[char] for char in x_data]
        
        
        y_data = np.asarray(y_data[1:-1].split(", "))
        y_data = [float(i) for i in y_data]
        
        
        X.append(np.asarray(integer_encoded))#.reshape(batch_size, time_steps))
        #print("X_train Shape = ", X_train[-1].shape)
        Y.append(np.asarray(y_data)) #.reshape(batch_size, category))
        #print("Y_train Shape = ", Y_train[-1].shape)
        
        count += 1

#h5f_X = h5py.File('X_train_superfam_all_bacteria_and_archaea.h5', 'w')
#h5f_X = h5py.File('X_test_1K_superfam_all_bacteria_and_archaea.h5', 'w')
h5f_X = h5py.File('X_train_superfam_16S_bacteria_and_archaea.h5', 'w')
h5f_X.create_dataset('X', data=X)
h5f_X.close()
#h5f_Y = h5py.File('Y_train_superfam_all_bacteria_and_archaea.h5', 'w')
#h5f_Y = h5py.File('Y_test_1K_superfam_all_bacteria_and_archaea.h5', 'w')
h5f_Y = h5py.File('Y_train_superfam_16S_bacteria_and_archaea.h5', 'w')
h5f_Y.create_dataset('Y', data=Y)
h5f_Y.close()
print("done")

done


In [25]:
print("hi")

hi


In [45]:
# Process test evaluation predictions from keras and associate them with labels
import numpy as np
import h5py

h5f = h5py.File('test_evaluation_Dec6_model_superfam_16S_bacteria_and_archaea.h5','r')
Y_Predictions = h5f['Y'][:]
h5f.close()

count = 0
file_write = gzip.open("test_evaluation_with_labels_Dec6_model_superfam_16S_bacteria_and_archaea_sequences_and_embeddings.gz", "wt")
with gzip.open("randomized_superfam_16S_bacteria_and_archaea_sequences_and_embeddings_test.gz", 'rt') as zip_file:
    for header in zip_file:
        #if (count >= 1000):
        #    break
        file_write.write(header.strip() + "\n")
        x_data = zip_file.readline().strip()
        y_data = zip_file.readline().strip()
        file_write.write(str(Y_Predictions[count].tolist()))
        file_write.write("\n")
        
        count += 1

file_write.close()

print("done")

done


In [46]:
# 2nd experiment: create the h5py files for the twin network training set
# translate the input file into three h5py files
import numpy as np
import h5py
import gzip
alphabet = "NACGT"
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
time_steps = 100
batch_size = 1
category = 10
count = 0
X_a = []
X_b = []
Y = []
with gzip.open("pairs_and_distance_16S_train.gz", 'rt') as zip_file:
    for header in zip_file:
        
        x_a_data = zip_file.readline().strip()
        x_b_data = zip_file.readline().strip()
        y_data = zip_file.readline().strip()
        integer_encoded_a = [char_to_int[char] for char in x_a_data]
        integer_encoded_b = [char_to_int[char] for char in x_b_data]
        
        print(y_data)
        y_data = np.asarray(float(y_data))
        #y_data = [float(i) for i in y_data]
        
        
        X_a.append(np.asarray(integer_encoded_a))#.reshape(batch_size, time_steps))
        X_b.append(np.asarray(integer_encoded_a))
        #print("X_train Shape = ", X_train[-1].shape)
        Y.append(np.asarray(y_data)) #.reshape(batch_size, category))
        #print("Y_train Shape = ", Y_train[-1].shape)
        
        count += 1

h5f_X = h5py.File('X_train_twin_sequences_a.h5', 'w')
h5f_X.create_dataset('X_a', data=X_a)
h5f_X.close()
h5f_X = h5py.File('X_train_twin_sequences_b.h5', 'w')
h5f_X.create_dataset('X_b', data=X_b)
h5f_X.close()
h5f_Y = h5py.File('Y_train_twin_sequences.h5', 'w')
h5f_Y.create_dataset('Y', data=Y)
h5f_Y.close()
print("done")

0.09215770096167437
0.09461050810386115
0.1030167773982367
0.10431462420509777
0.09571265856422773
0.09608603542546285
0.10069593657344775
0.10889019826887611
0.092564776554864
0.09158186681712786
0.10815210672537653
0.09868619184132288
0.13768293782351804
0.13312454698551962
0.13300088462876364
0.11776238244089252
0.06453318637679184
0.10348205991047074
0.09897456076271538
0.10290854774060712
0.11053929425541129
0.10425684395339604
0.10705003335865881
0.102103034813597
0.11231616002779626
0.13244317032200165
0.10250679907296512
0.09985101847780248
0.0964756048074841
0.05812626441155881
0.10117273722974213
0.10336060486458376
0.10485792434944279
0.11903692852341362
0.09454377273051659
0.10563452866987477
0.13362875143673925
0.09591502814444647
0.11649626421996237
0.10208971910540961
0.05981391505759591
0.09882683554639636
0.11189685251815944
0.10031666020692476
0.12124652971104574
0.09936266225666696
0.1068533829240827
0.0955850192943784
0.1382910933455578
0.11979732943523883
0.1087561

0.06134856018854113
0.056914212047456066
0.11849266097593103
0.08394852018390436
0.05928298726191435
0.11457081332749605
0.07382150672994886
0.08505169677843935
0.05218445343755094
0.07233961788792924
0.0798390334333087
0.0905957642092364
0.08744450032929979
0.11577919201820674
0.07791076201256893
0.045743056645012437
0.0874634845353427
0.09069088198740502
0.002366013741352224
0.06186439328344768
0.06507955958838721
0.038493659519013644
0.1016425571086336
0.10405030676809515
0.07511335876321906
0.08767073778988577
0.08268427395300473
0.0845431896269892
0.08711797501635976
0.0828517652781463
0.07988156083126821
0.08982200902861665
0.09130945241435773
0.10075720403258452
0.06991777915347289
0.11542408472078683
0.08611263974391296
0.06154440958368579
0.08287817518253333
0.08390331543231581
0.07248550228209324
0.015916783816014926
0.08275019838560842
0.13781602058464704
0.07021368973949632
0.08326574469276687
0.08367731808348348
0.07395704496817318
0.0749359648219933
0.06495411947099607
0.

0.09153847969718469
0.10758561152036489
0.09886200814341342
0.06046868687304419
0.06233293685102657
0.08129409599982185
0.07372494246028055
0.07732971052848797
0.07949972634422965
0.07671852124258075
0.07277920199750948
0.10281071654583165
0.07514951328333207
0.08048584254771607
0.07246127702075966
0.0983539901671984
0.10801997535078678
0.08276840342614705
0.07219015336340312
0.654508840254715
0.036787271398448805
0.09099572415358112
0.0765060369070622
0.07563168158966918
0.07170101340305927
0.0692822420787808
0.11146626666447462
0.10295846680612912
0.07131781912096777
0.0685899703510294
0.07598216389342341
0.08046512175172285
0.10176405309026056
0.10939427820769533
0.10668104305164763
0.07313845137854286
0.06910126483090347
0.07197720948705505
0.08800542777442512
0.09427042955716085
0.07047380275772591
0.06524931149274418
0.03827334859011639
0.07151824818058619
0.06749819941259705
0.1100945994553221
0.09162605931835738
0.1047553607145473
0.06611740667152638
0.07032184966899513
0.11056

0.07977798342876918
0.0845223082103951
0.07492881520848857
0.07507675144153403
0.07996156993949179
0.10788747813677069
0.09553720612182245
0.07954209107378449
0.07165369962657862
0.069511538363906
0.07408521494479571
0.08893679801632685
0.07923009945103253
0.07733097096202354
0.08617685676184678
0.07805411468680325
0.09764673365406051
0.06918248741164248
0.08852725418787993
0.07344923494079705
0.07728899386637533
0.07953621359786148
0.08394822386886866
0.0340820423979015
0.09913156510031679
0.10232326359062983
0.09476747173925547
0.07891728586721053
0.11285245193288188
0.06802891076043213
0.08440926078200096
0.08219745982772243
0.07029304178130145
0.06995423174914678
0.10560154055228388
0.0764544304836792
0.031798433500862214
0.07891856513445919
0.11593957873728902
0.08098459418431411
0.07448183707061909
0.10194341723090258
0.09902421338639529
0.08457370031024092
0.09870233638735934
0.0723916388883149
0.09184527190211442
0.07881741280485494
0.07572877827155294
0.1203830520788993
0.0767

0.07900352904632779
0.08409655927716905
0.04103749770500583
0.0862760648352123
0.03880333531381962
0.08811764006933384
0.07390378247007873
0.0972261163737117
0.09334821656244711
0.09837519603426814
0.1140967462479741
0.09819615840994383
0.027805787669137716
0.10764494550114724
0.09393895500291677
0.09902412740031066
0.13276446767219133
0.09146738454655641
0.10577861266237375
0.10362459753222567
0.10432783711223192
0.023087973269565368
0.09871122351253064
0.09992236865853482
0.09117922326183692
0.1040915732902245
0.09631712698735753
0.10825408116296374
0.10252500712388105
0.09113253139930788
0.10365545675092525
0.10130801496369018
0.09104203912287384
0.09178322383238041
0.11525544235665247
0.10452373679510077
0.10367850709918541
0.10641391625274646
0.09662479258785434
0.09870049253018057
0.06569276363675114
0.10435734399007233
0.09536780635834095
0.10095692719242838
0.10611044479699414
0.09647075633400118
0.1108666376198543
0.09302434372126456
0.10051464527644252
0.11480645243900692
0.0

...

0.08899604352890739
0.08260199661877443
0.016718163140881072
0.08006931452763898
0.08348614011137134
0.100168355386177
0.08212975335989936
0.09103541788454579
0.05612973154072422
0.1114254172726443
0.0809449448589179
0.08068782128538243
0.13410281914629954
0.09314007415506914
0.08968616316734107
0.021451266424087578
0.08888299610051324
0.09504394849633443
0.05831668385631819
0.08094366750440288
0.07931357729235394
0.08746344804665467
0.12054866909169598
0.09942144399995138
0.08651074657025924
0.08263460318460397
0.08252785000531554
0.09378960178957746
0.06445270210748556
0.09115904836459343
0.11280273854927048
0.08636547630527058
0.11884611070399712
0.1200621850202636
0.08389927655825002
0.0680552758698624
0.06048938139772989
0.07802323652123916
0.08993088569228441
0.07985863971564117
0.07624652812420168
0.0844392325018345
0.11628275700605603
0.08974067709226319
0.08223500375107781
0.027595165493539735
0.06633779696528852
0.08278599875156073
0.07802609559076176
0.06677944612255095
0.11

0.07575515383500606
0.07847987115155322
0.057710966258455745
0.08189053701411855
0.07666047386526975
0.07060584437151651
0.07644566326133673
0.10372990613634173
0.10621844236117213
0.06753385509622234
0.06647543240483905
0.06601272787573717
0.09935011795182518
0.08406805963052744
0.07058401425338315
0.07283105189508335
0.10128876209696286
0.06929403285757768
0.06344465755079061
0.10136030110533384
0.07323591874332092
0.061187957371752824
0.08517436047708381
0.09352867579216638
0.10539731320137356
0.06762947883155765
0.06916401355027237
0.0680691175489456
0.07029631465884205
0.06566477479835271
0.1002392638242583
0.07517066856118995
0.06433007247352959
0.07835974122887203
0.05511906070902639
0.06479340295546276
0.0661384230728586
0.06377973813782743
0.12464647241649798
0.06337190163722223
0.12482954569384277
0.03561623756714637
0.09757913498200445
0.09133703218355314
0.06971512059305528
0.06045095812455598
0.085124532308543
0.09422011256631421
0.10395393741970757
0.03290818194109095
0.0

0.1040094135252744
0.10043781766869422
0.12251723912970355
0.04891101452290739
0.10989530368743723
0.10116922563523342
0.044765461089171885
0.1101882774685506
0.11287874019980364
0.10798383187959341
0.09785067758552733
0.1165844340147101
0.10169026627573449
0.10167685486397106
0.09932367521135105
0.09627791981286318
0.11264825696246766
0.04605283118401525
0.10228400696353847
0.11782169419420996
0.09619113932798481
0.1029581634580494
0.1079791483172219
0.04589233448417272
0.11343595799691834
0.11001827976229941
0.11353857111142265
0.10209898533238346
0.19623265367603085
0.1530016365400106
0.10910795842600145
0.10138548758040315
0.10907038562513799
0.09589771254394452
0.1027761230735543
0.10938104426104046
0.04683232663640608
0.07361439279232093
0.10063378363645312
0.01959940106960634
0.11884619271653553
0.09753559477308071
0.11319954335654589
0.11841284674462244
0.10956856900880671
0.11247459345486323
0.10955153431092968
0.09835535062766738
0.09536215183529281
0.10140777937626952
0.1078

0.09941277651829733
0.10053017572075326
0.07666089445859384
0.09270477493205198
0.07347894487687608
0.11414703077217324
0.0824262691892998
0.07557708194303268
0.0840113364058444
0.09901758710406314
0.07807594324818351
0.08723130516543329
0.0811008430472005
0.05113401834748853
0.10846315784871308
0.09401650936359399
0.08068765607084016
0.08173013450403827
0.0984553620621953
0.07547337670675679
0.07676193401086287
0.08885154249229966
0.05261783379828228
0.05334774924727982
0.09011557505123298
0.0841859032869269
0.05105937534906854
0.0798134662345159
0.08054676908745434
0.04929552721589825
0.0774889243753448
0.07531572476906452
0.06265600534935477
0.08162624663019517
0.05313885259758139
0.08102163986766815
0.07413441032064508
0.08568157854545047
0.047076841576074474
0.07893540307120823
0.05081994459738004
0.10175262814216375
0.08972508704295339
0.07486601521775677
0.07538347354349306
0.048166613320155385
0.10592202203342838
0.04249198330171543
0.05502792605502238
0.08416199585476494
0.051

done
